In [9]:
import logging

from s2agr.builder import Builder
from s2agr.monitor import LoggingMonitor
from s2agr.persistence.database_catalogue import DatabaseCatalogue, production_connection
from s2agr.persistence.database_catalogue import count_rows

In [2]:
connection = production_connection()
catalogue = DatabaseCatalogue(connection)
librarian = Builder().with_catalogue(catalogue).with_monitor(LoggingMonitor(log_level=logging.DEBUG)).build()

In [3]:
query = """
select paper.paper_id, title from paper, author, wrote where author_name = 'Charles Duhigg' and
    paper.paper_id = wrote.paper_id and author.author_id = wrote.author_id
"""
papers = catalogue.query(query)
paper_ids = list(paper[0] for paper in papers)

In [4]:
paper_ids

['632218ee4d47598d24d816ebab841e56e10de3f6',
 '6b5461710f68e2740da8da05807b6779ef879f30',
 '770651a173ac74047a4219edab0b21f542566abd',
 'd0521d57fdce867d69dc18bc0d58fa9aaa03cc35',
 'e52eb167a5cfb9592f04ef910bc118470e6422e2',
 'eec14b112cbe570cefe6cb0ac5d760a86d818b42',
 'f14a947ff823488e10bc5bfcaa0812617b392a43',
 'f51507a0551786290d633fad87bd7d9a8d936db9']

In [5]:
papers = librarian.get_papers(*paper_ids)

In [6]:
papers

<generator object Librarian.get_papers.<locals>.<genexpr> at 0x7fd807696200>

In [7]:
for paper in papers:
    print(paper.title)
    librarian.add_citations_and_references(paper)

The Power of Habit: Why We Do What We Do in Life and Business
The Power of Habit: Why We Do What We Do, and How to Change
In Pursuit of Nissan, Jobs Lesson for the Tech Industry?
How Companies Learn Your Secrets
Smarter Faster Better: The Secrets of Being Productive in Life and Business
Dr. Elon & Mr. Musk
Annals of Epidemiology
28. In China, Human Costs Are Built Into an iPad


In [22]:

for paper_id in paper_ids:
    citing_ids = [row[0] for row in list(catalogue.query('select citing_id from citation where cited_id = (%s)', (paper_id,)))]
    # print(citing_ids)
    librarian.get_papers(*citing_ids)
    # print(paper_id, count_rows(connection.cursor(), 'cited_id', 'citation', paper_id))


In [21]:
connection.rollback()